Determinig the optimal number of hiidden layers and neurons for an ANN

In [2]:
#For hyper parameter tuning we sholud import scikeras library
import pandas as pd  
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
df=pd.read_csv("Churn_Modelling.csv")

In [4]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [7]:
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df["Gender"])

In [9]:
one_hot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=one_hot_encoder_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(["Geography"]))

In [10]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [11]:

df=pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [12]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
X=df.drop("Exited",axis=1)
y=df["Exited"]

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
import pickle
with open("HT_Label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("HT_Onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(one_hot_encoder_geo,file)

with open("HT_Scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [21]:
## Define a function to create the model and try different parameters(keras Classifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    #for remaining hidden layers
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

    return model 

In [22]:
## create the keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [23]:
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2,3],
    #'batch_size':[10,20],
    'epochs':[50,100]
}

In [24]:
#perform grid searchcv
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)

grid_result=grid.fit(X_train,y_train)

#print the best parameters
print("Best Parameters: ",grid_result.best_params_)
# print the best score
print("Best Score: ",grid_result.best_score_)

Epoch 1/50


d:\DL_NLP_Bootcamp\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\DL_NLP_Bootcamp\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7035 - loss: 0.5859
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8090 - loss: 0.4381
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8149 - loss: 0.4200
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8283 - loss: 0.4049
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8364 - loss: 0.3953
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8440 - loss: 0.3758
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8464 - loss: 0.3676
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8525 - loss: 0.3554
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8515 - loss: 0.3616
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8507 - loss: 0.3586
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8562 - loss: 0.3487
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy